In [ ]:
from sttn.data.lehd import OriginDestinationEmploymentDataProvider

provider = OriginDestinationEmploymentDataProvider()
network = provider.get_data(state='ny', year=2018)

In [ ]:
residence_sum = network.agg_adjacent_edges(aggs={'S000':'sum'}, outgoing=False).rename(columns={'S000':'total_jobs'})
with_residence = network.join_node_labels(residence_sum)
with_residence.plot(figsize=(15,15), column='total_jobs', cmap="Reds", scheme='quantiles', k=15, legend=True)

In [ ]:
NYC = ['New York County, NY', 'Queens County, NY','Kings County, NY','Bronx County, NY','Richmond County, NY']
# bay_area = ['San Francisco County, CA', 'San Mateo County, CA', 'Santa Clara County, CA', 'Alameda County, CA', 'Marin County, CA', 'Contra Costa County, CA']
nyc_network = network.filter_nodes(network.node_labels.county.isin(NYC))
nyc_residence_sum = nyc_network.agg_adjacent_edges(aggs={'S000':'sum'}, outgoing=False).rename(columns={'S000':'total_jobs'})
nyc_with_residence = nyc_network.join_node_labels(nyc_residence_sum)
nyc_with_residence.plot(figsize=(15,15), column='total_jobs', cmap="Reds", scheme='quantiles', k=20, legend=True)

In [ ]:
nyc_county_network = nyc_network.group_nodes('county')
nyc_county_network = nyc_county_network.agg_parallel_edges(column_aggs={'S000': 'sum'})
nyc_fdf = nyc_county_network.to_flow_date_frame(flow='S000')
nyc_fdf.plot_flows(flow_color='red')

In [ ]:
# Gravity model
from skmob.models import Gravity
conty_residence = nyc_county_network.agg_adjacent_edges(aggs={'S000': 'sum'}, outgoing=True).rename(columns={'S000': 'residence'})
county_outflow = nyc_county_network.agg_adjacent_edges(aggs={'S000': 'sum'}, outgoing=True, include_cycles=False).rename(columns={'S000': 'outflow'})
nyc_with_residence = nyc_county_network.join_node_labels(conty_residence).join_node_labels(county_outflow)
nyc_fdf = nyc_with_residence.to_flow_date_frame(flow='S000')

gravity_singly_fitted = Gravity(gravity_type='singly constrained')
gravity_singly_fitted.fit(nyc_fdf, relevance_column='residence')
print(Gravity(gravity_type='singly constrained'))
print(gravity_singly_fitted)

In [ ]:
# generate synthetics flows
import numpy as np
np.random.seed(0)

synth_fdf = gravity_singly_fitted.generate(nyc_with_residence.node_labels.reset_index(),
                                            tile_id_column='id',
                                            tot_outflows_column='outflow',
                                            relevance_column= 'residence',
                                            out_format='flows')
synth_fdf

In [ ]:
# Radiation model
from skmob.models import Radiation
radiation = Radiation()
np.random.seed(0)
rad_flows = radiation.generate(nyc_with_residence.node_labels.reset_index(), 
                                tile_id_column='id',  
                                tot_outflows_column='outflow', 
                                relevance_column='residence', 
                                out_format='flows')
rad_flows